# Transfer Learning - VGGNet

**Załadujemy gotowy model (w tym przypadku VGGNet19) i dostroimy go do nowego zadania klasyfikacji binarnej - przykład z książki "Uczenie głębokie i sztuczna inteligencja. Interaktywny przewodnik ilustrowany"**

### Ładujemy zależności

In [1]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Ładujemy wytrenowany model VGG19

In [2]:
vgg19 = VGG19(include_top=False, #ostatnie warstwy zagęszczone, specyficzne dla zbioru, nie zostają załadowane
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

80134624/80134624 [==============================] - 7s 0us/step


### "Zamrażamy" warstwy modelu VGGNet19 - nie będą się douczać

In [3]:
for layer in vgg19.layers:
    layer.trainable = False

In [4]:
vgg19

### Tworzymy całościowy model, "u góry" dodajemy VGGNet19, a dalej własne warstwy

**Zadanie 3. Dodaj warstwę spłaszczającą, dropout 0.5 i warstwę pozwalającą sklasyfikować binarnie obiekty**

In [5]:
model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 2)                 50178     
                                                                 
Total params: 20,074,562
Trainable params: 50,178
Non-trainable params: 20,024,384
_________________________________________________________________


### Kompilujemy model

In [7]:
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

### Pobierz zbiór danych z [Kaggle'a](https://www.kaggle.com/datasets/dansbecker/hot-dog-not-hot-dog) i rozpakuj

### Zbiór jest mały, więc tworzymy instancje klasy ImageDataGenerator, które pozwolą nam powiększyć dane

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

### Deklarujemy rozmiar paczki

In [9]:
batch_size = 32

### Powiększamy dane

In [10]:
train_generator = train_datagen.flow_from_directory(
    directory= 'archive/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory= 'archive/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


**Zadanie 4. Wypisz co oznaczają wszystkie zadeklarowane parametry generatorów danych**

<b>directory</b> - ścieżka w której znajdują się foldery klas
<br>
<b>target_size</b>  - rozmiar obrazów wchodzących, tj. zmienione do 224x224
<br>
<b>classess</b>  - dwie klasy daych, foldery w ścieżce zawierające zdjęcia hot dogów lub innych dań
<br>
<b>class_mode</b>  - deklarowany w zależności od ilości klas
<br>
<b>batch_size</b>  - liczba która dokładnie dzieli całkowitą liczbę obrazów
<br>
<b>shuffle</b>  - czy wymieszać kolejność plików
<br>
<b>seed</b>  - kolejny czynnik wpływający na randomowość obrazów poddawanych treningowi/walidacji

### Uczymy model z danymi z generatora (dla ułatwienia używamy do tego oddzielnej funkcji, choć będzie w następnej wersji tensorflowa usunięta)

In [11]:
model.fit_generator(train_generator, steps_per_epoch=15, epochs=16, validation_data=valid_generator, validation_steps=15)

C:\Users\angel\AppData\Local\Temp/ipykernel_16572/2768083069.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=15, epochs=16, validation_data=valid_generator, validation_steps=15)


Epoch 1/16
15/15 [==============================] - 108s 7s/step - loss: 1.2616 - accuracy: 0.5408 - val_loss: 1.3508 - val_accuracy: 0.5063
Epoch 2/16
15/15 [==============================] - 108s 7s/step - loss: 0.7543 - accuracy: 0.6888 - val_loss: 1.9330 - val_accuracy: 0.5021
Epoch 3/16
15/15 [==============================] - 108s 7s/step - loss: 0.8922 - accuracy: 0.6545 - val_loss: 0.5990 - val_accuracy: 0.7042
Epoch 4/16
15/15 [==============================] - 108s 7s/step - loss: 0.4735 - accuracy: 0.7854 - val_loss: 0.4511 - val_accuracy: 0.7792
Epoch 5/16
15/15 [==============================] - 108s 7s/step - loss: 0.6344 - accuracy: 0.7296 - val_loss: 0.7820 - val_accuracy: 0.6854
Epoch 6/16
15/15 [==============================] - 109s 7s/step - loss: 0.4061 - accuracy: 0.8112 - val_loss: 0.9354 - val_accuracy: 0.6479
Epoch 7/16
15/15 [==============================] - 108s 8s/step - loss: 0.6905 - accuracy: 0.7189 - val_loss: 0.4748 - val_accuracy: 0.7812
Epoch 8/16
15